# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합 - pipeline**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * 단위 테스트
        * 각 기능(함수)에 대해 단계별로 테스트를 수행하며 오류를 해결합니다.
    * 파이프라인 구축
        * 단계1의 결과가 단계2 모델에 input이 되고, 모델의 예측 결과를 기반으로
        * 응급실 추천되도록
        * 조원들이 녹음한 음성 파일에 임의의 좌표(위도, 경도)값을 부여
            * 음성파일 이름과 좌표를 저장하는 별도 데이터셋 생성
        * 각 모듈을 연결하여 파이프라인 구성하는 ipynb 파일 생성



## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [5]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [6]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
from IPython.display import display
import json

# path = '/content/drive/MyDrive/project6_2/'
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 조에서 생성한 모듈 불러오기 -------------
from emergency import set_api_key, transcribe_audio, text2summary
from emergency import severity_prediction, recommend_hospital3


#### 3) 경로 설정 및 데이터 불러오기

In [ ]:
# 파일 경로
api_key_file = path + 'api_key.txt'
save_directory = path + 'fine_tuned_bert/'
map_key_file = path + 'map_key.txt'

# 응급실 정보 데이터
data = pd.read_csv(path+'응급실정보.csv')

# 입력 환자 정보
audio_file = path + 'audio/audio1.mp3' # 환자 오디오 파일 경로
lat, lng = [37.2873665, 126.9912075] # 환자 위치 정보

## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

### (1) open ai key 등록

In [ ]:
# OPEN API KEY 환경변수 설정
set_api_key(api_key_file)


### (2) audio to text

In [ ]:
# 오디오를 텍스트로 변환
input_text = transcribe_audio(audio_file)

print(input_text)

### (3) text summary

In [ ]:
# 텍스트를 요약
text = text2summary(input_text)

print(text)

### (4) 응급실 등급분류

In [ ]:
# 모델 예측
severity_class = severity_prediction(text, save_directory) # default: verbose=False

print(severity_class)

### (5) 응급실추천

In [ ]:
# 가까운 병원 3개 출력
if severity_class <= 3: # 중증도가 1, 2, 3등급일 경우에만 추천 진행
    df = recommend_hospital3(lat, lng, map_key_file, data=data).copy() # alpha: 기본값 lat, lng = [0.1, 0.1]
    print(f'환자 위치: [{lat},{lng}]')
    display(df)
else:
		print('코멘트: 빠른 시일내에 인근 병원에 방문하세요!')

## **3. 파이프라인**

* 세부사항
    * [2. 단계별 테스트] 의 내용을 순차적으로 정리합니다.
        * 데이터 처리 전 준비작업 : 한번 실행하면 되는 영역
            * 키, 데이터로딩
            * 모델/토크나이저 로딩
        * 입력값이 들어 왔을 때 출력값까지 처리되는 영역

In [ ]:
# 파일 경로
api_key_file = path + 'api_key.txt'
save_directory = path + 'fine_tuned_bert/'
map_key_file = path + 'map_key.txt'

# 응급실 정보 데이터
data = pd.read_csv(path+'응급실정보.csv')

# 입력 환자 정보
audio_file = path + 'audio/audio1.mp3' # 환자 오디오 파일 경로
lat, lng = [37.2873665, 126.9912075] # 환자 위치 정보

In [13]:
def setup_pipeline(path):
    """
    파이프라인 실행 전 필요한 준비작업을 수행하는 함수

    Args:
        path (str): 프로젝트 기본 경로

    Returns:
        dict: 파이프라인에 필요한 모든 설정값들을 담은 딕셔너리
    """
    # 경로 설정
    api_key_file = path + 'api_key.txt'
    save_directory = path + 'fine_tuned_bert/'
    map_key_file = path + 'map_key.txt'

    # OPEN API KEY 환경변수 설정
    set_api_key(api_key_file)

    # 응급실 정보 데이터 로드
    data = pd.read_csv(path+'응급실정보.csv')

    # 설정값들을 딕셔너리로 반환
    config = {
        'save_directory': save_directory,  # 모델 저장 경로만 전달
        'map_key_file': map_key_file,
        'hospital_data': data
    }

    return config

def emergency_pipeline(audio_file, patient_location, config, verbose=False):
    """
    응급상황 처리를 위한 메인 파이프라인 함수

    Args:
        audio_file (str): 음성 파일 경로
        patient_location (list): [위도, 경도]
        config (dict): setup_pipeline에서 반환된 설정값들
        verbose (bool): 상세 출력 여부

    Returns:
        dict: 파이프라인 처리 결과
    """
    try:
        # 1. 음성을 텍스트로 변환
        if verbose: print("1. 음성을 텍스트로 변환 중...")
        input_text = transcribe_audio(audio_file)

        # 2. 텍스트 요약
        if verbose: print("2. 텍스트 요약 중...")
        summary_text = text2summary(input_text)

        # 3. 중증도 예측
        if verbose: print("3. 중증도 예측 중...")
        severity = severity_prediction(summary_text, config['save_directory'])  # 경로만 전달

        # 4. 병원 추천 (중증도가 3 이하인 경우만)
        hospital_recommendations = None
        if severity <= 3:
            if verbose: print("4. 인근 병원 검색 중...")
            lat, lng = patient_location
            hospital_recommendations = recommend_hospital3(
                lat, lng,
                config['map_key_file'],
                config['hospital_data']
            )

        # 결과를 딕셔너리로 정리
        result = {
            'original_text': input_text,
            'summary_text': summary_text,
            'severity_class': severity,
            'patient_location': patient_location,
            'hospital_recommendations': hospital_recommendations
        }

        return result

    except Exception as e:
        print(f"Error in pipeline: {str(e)}")
        return None

# 결과 출력 함수 추가
def print_pipeline_results(result):
    """
    파이프라인 결과를 출력하는 함수

    Args:
        result (dict): emergency_pipeline의 결과
    """
    if result is None:
        print("파이프라인 처리 중 오류가 발생했습니다.")
        return

    print("\n=== 처리 결과 ===")
    print(f"원본 텍스트: {result['original_text']}")
    print(f"요약 텍스트: {result['summary_text']}")
    print(f"중증도 등급: {result['severity_class']}")

    if result['hospital_recommendations'] is not None:
        print("\n=== 추천 병원 ===")
        print(f"환자 위치: {result['patient_location']}")
        display(result['hospital_recommendations'])
    else:
        print('\n코멘트: 빠른 시일내에 인근 병원에 방문하세요!')

In [12]:
# 1. 초기 설정
path = '/content/drive/MyDrive/project6_2/'
config = setup_pipeline(path)

# 2. 파이프라인 실행
audio_file = path + 'audio/audio1.mp3'
patient_location = [37.2873665, 126.9912075]

result = emergency_pipeline(audio_file, patient_location, config, verbose=True)

# 3. 결과 출력
print_pipeline_results(result)

1. 음성을 텍스트로 변환 중...
2. 텍스트 요약 중...
3. 중증도 예측 중...
4. 인근 병원 검색 중...

=== 처리 결과 ===
원본 텍스트: 지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.

요약 텍스트: 아빠가 계단에서 넘어져 머리에서 피를 흘리고 숨은 쉬고 어지러워하는 상황. 응급실 방문이 필요할 수 있음. 아빠, 넘어짐, 피, 머리, 응급실
중증도 등급: 2

=== 추천 병원 ===
환자 위치: [37.2873665, 126.9912075]


,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,dist,time
0,경기도의료원수원병원,경기도 수원시 장안구 수성로245번길 69 (정자동),지역응급의료기관,031-888-0114,031-888-0119,37.291882,126.996388,1.59,8분
1,수원중앙병원,경기도 수원시 권선구 권선로 654 (권선동),응급실운영신고기관,031-229-9777,031-229-9888,37.259913,127.022841,6.65,17분
2,가톨릭대학교성빈센트병원,경기도 수원시 팔달구 중부대로 93 (지동),지역응급의료센터,031-1577-8588,031-249-7370,37.277922,127.027449,6.70,19분
